<a href="https://colab.research.google.com/github/KendallScott/Machine-Learning/blob/main/forestCoverLogReg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import packages

import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import ShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn import metrics as mt
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

pd.set_option('display.max_columns',None)

## Reading and cleaning the data

In [ ]:
forest_cover_type = pd.read_csv('/Users/taddbackus/School/spring23/ML1/Lab1/covtype.csv')
forest_cover_type.head()

Saving covtype.csv to covtype.csv


### Renaming wilderness areas


In [ ]:
forest_cover_type.rename(columns={'Wilderness_Area1':'Rawah','Wilderness_Area2':'Neota','Wilderness_Area3':'Comanche Peak','Wilderness_Area4':'Cache la Poudre'},inplace=True)
forest_cover_type.head()

(581012, 55)


,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5


## Creating a new dataset that only contains the most common cover types


In [ ]:
forestDF = forest_cover_type[(forest_cover_type['Cover_Type']==1) | (forest_cover_type['Cover_Type']==2)]


### Renaming cover type to actual names
We could skip the next couple steps and just leave cover type as 1 and 2 and everything should work. Left it like this for workflow/reference.

In [ ]:
# this isnt necessary but maybe helpful for plotting or seeing which cover types are use
forestDF = forestDF.astype({'Cover_Type':'string'})
forestDF['Cover_Type'] = forestDF['Cover_Type'].str.replace('1','Spruce/Fir')
forestDF['Cover_Type'] = forestDF['Cover_Type'].str.replace('2','Lodgepole Pine')
forestDF['Cover_Type'].describe().transpose()

### Coding Lodgepole Pine as 1 and Spruce/Fir as 0


In [ ]:
# Lodgepole Pine = 1 | Spruce/Fir = 0
forestDF['treeType'] = forestDF.Cover_Type=='Lodgepole Pine'
forestDF.treeType = forestDF.treeType.astype(np.int64)

### Deleting original cover type column


In [ ]:
if 'Cover_Type' in forestDF:
    del forestDF['Cover_Type']

## Logistic Regression
Splitting the data between response and predictors


In [ ]:
if 'treeType' in forestDF:
    Y = forestDF['treeType'].values
    del forestDF['treeType']
    X = forestDF.values

### Creating cross validation object


In [ ]:
lr_clf = LogisticRegression(penalty='l2',C=1.0,class_weight=None)


In [ ]:
## Running logistic regression on 3 random test/train splits


In [ ]:
iter_num = 0

for iter_num, (train_indices,test_indices) in enumerate(cv_object.split(X,Y)):
    lr_clf.fit(X[train_indices],Y[train_indices])
    y_hat = lr_clf.predict(X[test_indices])
    
    print('****Iteration',iter_num,'****')
    print('Accuracy',mt.accuracy_score(Y[test_indices],y_hat))
    print('Confusion Matrix \n',mt.confusion_matrix(Y[test_indices],y_hat))

## Looking at weights of model


In [ ]:
weights = lr_clf.coef_.T
varNames = forestDF.columns
for coef, name in zip(weights,varNames):
    print(name,'has weight of',coef[0])

## Scaling the weights for interpretability


In [ ]:
sclObj = StandardScaler()
sclObj.fit(X[train_indices])

X_trainScaled = sclObj.transform(X[train_indices])
x_testScaled = sclObj.transform(X[test_indices])

lrs_clf = LogisticRegression(penalty='l2',C=0.05)
lrs_clf.fit(X_trainScaled,Y[train_indices])

y_hatS = lrs_clf.predict(x_testScaled)

acc = mt.accuracy_score(Y[test_indices],y_hatS)
conf = mt.confusion_matrix(Y[test_indices],y_hatS)
print('accuracy:',acc)
print(conf)

zipVars = zip(lrs_clf.coef_.T,forestDF.columns)
zipVars = sorted(zipVars)

for coef, name in zipVars:
    print(name,'has weight of', coef[0])

### Plotting weights because probably useful (scaled weights)


In [ ]:
%matplotlib inline
plt.style.use('ggplot')
weights = pd.Series(lrs_clf.coef_[0],index=forestDF.columns)
weights.plot(kind='bar')
plt.show()

### Support Vector Machine
Creating and running the SVM using the scaled and split sets from earlier
do not run, it isnt finishing for some reason and breaks the whole notebook

In [ ]:
#svm_clf = SVC(C=0.5, kernel='rbf', degree=3, gamma='auto')
#svm_clf.fit(X_trainScaled,Y[train_indices])

In [ ]:
#y_hatSVM = svm_clf.predict(X_testScaled)

#acc = mt.accuracy_score(Y[test_indices],y_hatSVM)
#conf = mt.confusion_matrix(Y[test_indices],y_hatSVM)
#print('accuracy:',acc)
#print(conf)